Try out spacy dependency parser

In [1]:
import spacy
from spacy import displacy

combine all the dataset into one single file

In [38]:
import pandas as pd

In [77]:
nlp = spacy.load("en_core_web_lg")

In [80]:
# text="The price and features more than met my needs."
# text="The screen shows great colors."
# text = "It has good speed and plenty of hard drive space."
text = "I charge it at night and skip taking the cord with me because of the good battery life."
doc = nlp(text)
displacy.render(doc, style="dep")

#### These are the rules with high precision

In [ ]:
def apply_extraction(review_body, nlp):
    # review_body = row["Sentence"]
    # review_id = row["id"]
    
    doc = nlp(review_body)
            
    # rule1 - Ex: The phone is very lightweight to carry.
    r1_pairs = []
    for token in doc:
        A = ""
        M = ""
        for child in token.children:
            if child.dep_ == "nsubj" and child.pos_ == "NOUN":
                A = child.text
                
                if not M and child.dep_ == "dobj" and child.pos_ != "NOUN":
                    M = child.text

                if not M and child.dep_ == "acomp":
                    M = child.text
        if A and M:
            r1_pairs.append((A, M))
            
    # rule1
    r2_pairs = []
    for token in doc:
        if token.dep_ in ("amod", "advmod") and token.head.pos_ == "NOUN":
            A = token.head.text
            # check if it is a noun phrase or not
            for c in token.head.children:
                if c.pos_ == "NOUN" and c.dep_ == "compound":
                    A = c.text + " " + A
                    M = token.text
                if A and M:
                    r2_pairs.append((A, M))
            
    # rule3
    r3_pairs = []
    # prev_token_pos = ""
    for token in doc:
        A = ""
        M = ""
        for child in token.children:
            if token.pos_ == "NOUN" and child.dep_ == "prep":
                A = token.text
                M = child.text
            # if prev_token_pos != "NOUN" and token.dep_ =="prep" and token.pos_ == "ADP" and child.dep_ == "pobj":
            #     A = child.text
        if A and not M:
            r3_pairs.append((A, M))
        # prev_token_pos = token.pos_

            
    # rule4
    r4_pairs = []
    for token in doc:
        children = token.children
        A = ""
        M = ""
        for child in children:
            if child.dep_ == "nsubjpass":
                A = child.text
                
            if child.dep_ == "advmod":
                M = child.text
        
        if A and M:
            r4_pairs.append((A, M))
            
    # rule5
    r5_pairs = []
    for token in doc:
        children = token.children
        A = ""
        M = ""
        for child in children:
            if child.dep_ == "nsubj":
                A = child.text
                
            if child.dep_ == "cop":
                M = child.text
        
        if A and M:
            r5_pairs.append((A, token.text))
            
    # rule 6 - Ex. I like the lens of the screen.
    r6_pairs = []
    for token in doc:
        A = ""
        M = ""
        if token.pos_ == "VERB":
            for child in token.children:
                if child.dep_ == "nsubj" and child.pos_ == "PRON":
                    continue
                if child.dep_ == "dobj" and child.pos_ == "NOUN":
                    phrase1 = child.text
                    # check if it is a noun phrase or not
                    for c in child.children:
                        if c.pos_ == "NOUN" and c.dep_ == "compound":
                            A = c.text + " " + phrase1
                            M = token.text
        if A and M:
            r6_pairs.append((A, M))
                
                
    # rule 7 - I would like to comment on the camera of this phone.
    A = ""
    M= ""
    r7_pairs=[]
    for i in range(len(doc)-1):
        if doc[i].pos_ == "VERB" and doc[i+1].pos_ == "ADP" and doc[i+1].dep_ == "prep":
            for token in doc[i+1:]:
                for child in token.children:
                    if child.dep_ == "pobj" and child.pos_ == "NOUN":
                        A = child.text
                        M = doc[i].text
                break   
        if A and M:
            r7_pairs.append((A, M))
            break
            
    # rule 8 - It is easy to use
    r8_pairs = []
    for token in doc:
        A = ""
        M = ""
        if token.pos_ == "AUX" and len([child for child in token.children]) >= 2:
            for child in token.children:
                if child.dep_ == "acomp" and child.pos_ == "ADJ":
                    M = child.text
                if child.dep_ == "xcomp" and child.pos_ in ("VERB", "NOUN"):
                    A = child.text
            if A and M:
                r8_pairs.append((A, M))

            
    # aspects = []
    # print(r1_pairs, r2_pairs, r3_pairs, r4_pairs, r5_pairs, r6_pairs, r7_pairs, r8_pairs)
    aspects_pairs = r1_pairs + r2_pairs + r3_pairs + r4_pairs + r5_pairs + r6_pairs + r7_pairs + r8_pairs
    # aspect_dict = {"review_id": review_id, "review_body": review_body, "aspect_pairs": aspects}
    return list(set(aspects_pairs))

In [13]:
for token in doc:
    if not token.children:
        print(token, token.pos_, token.dep_)
    else:
        for word in token.children:
            if word.pos_ in ["NOUN", "VERB"]:
                print(word, word.pos_, word.dep_)

ignored VERB conj
waitstaff NOUN pobj


In [16]:
doc=nlp("The company produces a good movie")
noun_adj_pairs = []
for i,token in enumerate(doc):
    if token.pos_ not in ('NOUN','PROPN'):
        continue
    for j in range(i+1,len(doc)):
        if doc[j].pos_ == 'ADJ':
            noun_adj_pairs.append((token,doc[j]))
            break
print(noun_adj_pairs)
displacy.render(doc, style="dep")

[(company, good)]


In [19]:
text="We were seated and ignored by waitstaff."
doc = nlp(text)
rule1_pairs = []
for token in doc:
    if token.dep_ == "amod":
        rule1_pairs.append((token.head.text, token.text))

In [20]:
rule1_pairs

[]

In [21]:
text="We were seated and ignored by waitstaff."
doc = nlp(text)

In [26]:
import pandas as pd

In [27]:
file_path = "datasets/Restaurant_reviews/Restaurants_Train_v2.csv"
# raw_data = pd.read_table(file_path, error_bad_lines=False, nrows=100)
raw_data = pd.read_csv(file_path)
reviews = raw_data[["id", "Sentence"]]
reviews.head()

,id,Sentence
0,3121,But the staff was so horrible to us.
1,2777,"To be completely fair, the only redeeming fact..."
2,1634,"The food is uniformly exceptional, with a very..."
3,1634,"The food is uniformly exceptional, with a very..."
4,1634,"The food is uniformly exceptional, with a very..."


In [4]:
def visualize_parser(text):
    return displacy.render(nlp(text), style="dep")

In [53]:
def find_nouns(text):
    r1 = []
    for token in nlp(text):
        # r2 = []
        if token.dep_ == "amod":
            print(r1)
            r1.append((token.head.text, token.text))
        # if token.pos_ in ["NOUN", "PROPN"]:
        #     r2 = [i for i in token.children if i.pos_ in ["ADJ", "NOUN", "PROPN"]]
            # if r2:
                # return {"aspect": token, "opinion word": comps}
    # return (r1, r2)
    return r1

In [12]:
visualize_parser("The camera is amazing and easy to use.")

In [15]:
visualize_parser("We ordered the chicken casserole, but what we got were a few small pieces of chicken, all dark meat and on the bone")

In [ ]:
example = ["Love the sleekness of the player.", 
           "Faan's got a great concept but a little rough on the delivery."]

In [145]:
# if token = noun and it has child with dependency of preposition --> then the token is noun
r_pairs = []
prev_token_pos = ""
for token in nlp("But the staff was so horrible to us."):
    print(token, [i for i in token.children])
    A = ""
    M = ""
    for child in token.children:
        if token.pos_ == "NOUN" and child.dep_ == "prep":
            A = token.text
        # if prev_token_pos != "NOUN" and token.dep_ =="prep" and token.pos_ == "ADP" and child.dep_ == "pobj":
        #     A = child.text
    if A and not M:
        r_pairs.append((A, M))
    prev_token_pos = token.pos_
r_pairs

But []
the []
staff [the]
was [But, staff, horrible, .]
so []
horrible [so, to]
to [us]
us []
. []


[('us', '')]

In [147]:
r2_pairs = []
for token in nlp("But the staff was so horrible to us"):
    print(token, [i for i in token.children])
    A = ""
    M = ""
    for child in token.children:
        if child.dep_ == "nsubj":
            A = child.text

        if not M and child.dep_ == "dobj":
            M = child.text
        if not M and child.dep_ == "acomp":
            M = child.text
    if A and M:
        r2_pairs.append((A, M))
print(r2_pairs)

But []
the []
staff [the]
was [But, staff, horrible]
so []
horrible [so, to]
to [us]
us []
[('staff', 'horrible')]


In [143]:
r2_pairs

[('rice', 'amazing')]

In [105]:
r1=[]
for token in nlp("To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora."):
    if token.dep_ == "amod" and token.pos_ == "ADJ" and token.head.pos_ == "NOUN":
        r1.append((token.head.text, token.text))
    # if token.children:
    #     for word in token.children:
    #         if word.dep_ == "amod":
    #             r1.append((word.head.text, word.text))
print(r1)

[('factor', 'only'), ('deficiencies', 'other')]


In [54]:
reviews = reviews[:10]
for row in reviews.itertuples():
    print(row.Sentence)
    result = find_nouns(row.Sentence)
    print("results: ", result)

But the staff was so horrible to us.
results:  []
To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.
[]
[('factor', 'only')]
[('factor', 'only'), ('factor', 'redeeming')]
[('factor', 'only'), ('factor', 'redeeming'), ('above', 'average')]
results:  [('factor', 'only'), ('factor', 'redeeming'), ('above', 'average'), ('deficiencies', 'other')]
The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.
[]
results:  [('kitchen', 'capable')]
The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.
[]
results:  [('kitchen', 'capable')]
The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.
[]
results:  [('k